# Projeto 1 - Ciência dos Dados

Nome: Luca Coutinho Melão

Nome: Thiago de Donato Youssef

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [43]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [44]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [45]:
train = pd.read_excel(filename)
train.head(20)

,Treinamento,Relevância
0,a sony anunciando o preço do ps5 hoje\nhttps:/...,0
1,vou prar as minhas atividades de desenvolvimen...,0
2,alguma live do evento do ps5 com tradução em p...,1
3,"@thecure70s pô pior que n sei, mas vai ser dep...",1
4,@fiaspo olha... e se rolar gameplay de tlou 2 ...,1
5,espero que o ps5 saia por 10 reais e os jogos ...,0
6,"hoje é dia, né?\nquest, ps5, hero9... \ncrysis...",1
7,#ps5 daqui a pouco saberemos o preço oficial 😬...,0
8,essa dlc de spider man tá linda #ps5,1
9,"n tá com cara de ser o ffxvi não, tá meio basi...",0


In [46]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Relevância
0,daqui a meia hora tem novidades do #ps5! ansio...,1
1,"@smashrs to plugado, ansioso pela decepção em ...",1
2,"eu: n vou ver o ps5 showcase, tenho muita cois...",0
3,será que vai rolar o preço do ps5? mesmo saben...,1
4,rt @ojovemgamer: quando a sony divulgar o preç...,0
...,...,...
348,"""preço e data o que vier é lucro"" kkkkkkk\n#ps...",1
349,bora ver o preço do ps5 juntos? https://t.co/...,0
350,rt @thiagomoreiracs: a única coisa que eu espe...,1
351,"se tiver god of war ""2"" nesse evento do ps5, e...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O PS5 É O NOVO CONSOLE DE JOGOS DA SONY, COM SEU LANÇAMENTO AO FINAL DE 2020.

RELEVANTE

Tweets que expressam o sentimento do usuário com relação ao console, remetem geralmente às expectativas sobre os preços, jogos mais aguardados, comparações com a concorrência.

IRRELEVANTE

Tweets que não estão relacionados ao produto, ou que não agregam nada à análise de sentimentos sobre ele. Tais como: anúncios de lives e comentários sem significado.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [47]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

# Dicionarios que guardam quantas vezes uma palavra apareceu na base de dados de acordo com a sua relevância

def dicionario(dados,base):
    a = 0
    b = 0
    
    for tweet in dados.loc[dados[base]==0, base]:
        tweet = cleanup(tweet)
        words = tweet.split()
        for word in words:
            if word not in dc_0.keys():
                dc_0[word] = 1
            else:
                dc_0[word] += 1
                
    for tweet in dados.loc[dados[base]==1, base]:
        tweet = cleanup(tweet)
        words = tweet.split()
        for word in words:
            if word not in dc_1.keys():
                dc_1[word] = 1
            else:
                dc_1[word] += 1
    
    for i in dc_0.values():
        a += i
    for i in dc_1.values():
        b += i
        
    den = len(dc_1)
    repetido = False
    
    for i in dc_0.keys():
        for x in dc_1.keys():
            if i == x:
                repetido = True
        if repetido == False:
            den += 1
            
    den_0 = a+den
    den_1 = b+den
            
    return [dc_0,dc_1,den_0,den_1]

# Função que calcula a probabilidade de um tweet não ser relevante
def calcula_prob_0(tweet,dic):
    dc_0 = dic[0]
    den = dic[2]
    prob = 1
    words = tweet.split()
    for word in words:
        if word in dc_0.keys():
            prob *= ((dc_0[word]+1)/den)
        else:
            prob *= ((1)/den)
    return prob

# Função que calcula a probabilidade de um tweet ser relevante
def calcula_prob_1(tweet,dic):
    dc_1 = dic[1]
    den = dic[3]
    prob = 1
    words = tweet.split()
    for word in words:
        if word in dc_1.keys():
            prob *= ((dc_1[word]+1)/den)
        else:
            prob *= ((1)/den)
    return prob

# Função que classifica os tweets
def classificador(dados,base):
    dic = dicionario(dados,base)
    for tweet in dados[base]:
        tweet = cleanup(tweet)
        P_T_dado_R = calcula_prob_1(tweet,dic)
        P_T_dado_Rc = calcula_prob_0(tweet,dic)
        if (P_T_dado_R > P_T_dado_Rc):
            dados.loc[(dados[base]==tweet),'Relevância_Teste'] = '1'
        else:
            dados.loc[(dados[base]==tweet),'Relevância_Teste'] = '0'

# Classificando base de testes
classificador(test,'Teste')


In [49]:
test

,Teste,Relevância,Relevância_Teste
0,daqui a meia hora tem novidades do #ps5! ansio...,1,NaN
1,"@smashrs to plugado, ansioso pela decepção em ...",1,NaN
2,"eu: n vou ver o ps5 showcase, tenho muita cois...",0,NaN
3,será que vai rolar o preço do ps5? mesmo saben...,1,NaN
4,rt @ojovemgamer: quando a sony divulgar o preç...,0,NaN
...,...,...,...
348,"""preço e data o que vier é lucro"" kkkkkkk\n#ps...",1,NaN
349,bora ver o preço do ps5 juntos? https://t.co/...,0,NaN
350,rt @thiagomoreiracs: a única coisa que eu espe...,1,NaN
351,"se tiver god of war ""2"" nesse evento do ps5, e...",1,NaN


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [50]:
test

,Teste,Relevância,Relevância_Teste
0,daqui a meia hora tem novidades do #ps5! ansio...,1,NaN
1,"@smashrs to plugado, ansioso pela decepção em ...",1,NaN
2,"eu: n vou ver o ps5 showcase, tenho muita cois...",0,NaN
3,será que vai rolar o preço do ps5? mesmo saben...,1,NaN
4,rt @ojovemgamer: quando a sony divulgar o preç...,0,NaN
...,...,...,...
348,"""preço e data o que vier é lucro"" kkkkkkk\n#ps...",1,NaN
349,bora ver o preço do ps5 juntos? https://t.co/...,0,NaN
350,rt @thiagomoreiracs: a única coisa que eu espe...,1,NaN
351,"se tiver god of war ""2"" nesse evento do ps5, e...",1,NaN


In [16]:
print(x)

None


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**